# Predicting Daily Curtailment Events

Process Overview:

1.  Label curtailment events (i.e. define a decision boundary)
2.  Partition historic data into training and test sets
3.  Fit a statistical model to the training data
4.  Predict against the test data
5.  Evaluate the performance of the model against known labels in the test data.


Models:
- `C(curtailment_event)`
- TODO: Pending multi-year weather pipeline `C() ~ T + I + month*load`

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

from src.conf import settings

In [2]:
df = pd.concat(
    [
        pd.read_parquet(settings.DATA_DIR / f"processed/caiso/{y}.parquet") for y in range(2017,2020)
    ]
)
df.columns = df.columns.str.lower().str.replace(" ", "_")
columns = ["load", "net_load", "solar_curtailment", "solar"]
df = df[columns].groupby(pd.Grouper(freq="D")).sum()
df.reset_index(inplace=True)

In [3]:
df.head()

,timestamp,load,net_load,solar_curtailment,solar
0,2017-01-01 00:00:00+00:00,4.065994e+06,3.161397e+06,26760.716117,497133.285691
1,2017-01-02 00:00:00+00:00,6.690988e+06,5.889257e+06,43.705500,206467.221000
2,2017-01-03 00:00:00+00:00,7.231820e+06,6.737002e+06,54.841500,245951.093081
3,2017-01-04 00:00:00+00:00,7.368107e+06,6.683064e+06,20.247000,359225.032181
4,2017-01-05 00:00:00+00:00,7.245502e+06,6.373406e+06,190.026216,342741.964986


## Seasonally-driven Model

The goal of this model is to define a naive threshold that captures "significant" curtailment events.  From the EDA, we observed (very roughly) that large curtailment events could be captured by comparing a ratio of curtailment amount to solar output.  One mechanistic explanation for this could be that curtailment is most pronounced when load usage is low, but solar resource is high (e.g. temperate weather in population centers coinciding with clear sunny days.)

In [4]:
# Label Data - based on our EDA, we might start by "guessing" a threshold of importance of .05
# Later methods will be less biased, and allow for more variance.
# TODO: Try to find natural clusterings through an unsupervised process to label the dataset, and try to predict those labels.
df["curtailment_event"] = pd.Categorical(df["solar_curtailment"]/df["solar"] > .05)

df["is_weekday"] = pd.Categorical(df["timestamp"].dt.weekday.isin([5, 6]))

In [5]:
training_data = df.query("timestamp.dt.year < 2019")
test_data = df.query("timestamp.dt.year == 2019")

We hope to motivate a few basic expectations about this model.

1.  Show that seasonal variation (captured through simple time dependence) can perform better than guessing randomly.
2.  That seasonal variation alone is not sufficient to perform useful metrics

### Logistic Model

In [6]:
model = "C(curtailment_event) ~ C(timestamp.dt.month) + C(is_weekday) + load"
result = smf.glm(
    model,
    training_data,
    family=sm.families.Binomial()
).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       Generalized Linear Model Regression Results                                       
=========================================================================================================================
Dep. Variable:     ['C(curtailment_event)[False]', 'C(curtailment_event)[True]']   No. Observations:                  730
Model:                                                                       GLM   Df Residuals:                      716
Model Family:                                                           Binomial   Df Model:                           13
Link Function:                                                             logit   Scale:                          1.0000
Method:                                                                     IRLS   Log-Likelihood:                -151.66
Date:                                                           Thu, 12 Mar 2020   Deviance:                       303.32
Time:                                                                   04:15:22   Pearson chi2:                     683.
No. Iterations:                                                               24                                         
Covariance Type:                                                       nonrobust                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     -13.6675      4.145     -3.297      0.001     -21.792      -5.543
C(timestamp.dt.month)[T.2]     -1.0978      0.708     -1.551      0.121      -2.485       0.289
C(timestamp.dt.month)[T.3]     -1.6636      0.676     -2.462      0.014      -2.988      -0.339
C(timestamp.dt.month)[T.4]     -0.6237      0.715     -0.873      0.383      -2.024       0.777
C(timestamp.dt.month)[T.5]     -0.9646      0.725     -1.331      0.183      -2.385       0.456
C(timestamp.dt.month)[T.6]     -0.0821      1.215     -0.068      0.946      -2.463       2.299
C(timestamp.dt.month)[T.7]     18.3464    2.3e+04      0.001      0.999    -4.5e+04     4.5e+04
C(timestamp.dt.month)[T.8]     18.2086   2.44e+04      0.001      0.999   -4.78e+04    4.78e+04
C(timestamp.dt.month)[T.9]     -0.5793      1.247     -0.465      0.642      -3.023       1.865
C(timestamp.dt.month)[T.10]    -1.2986      0.742     -1.750      0.080      -2.753       0.156
C(timestamp.dt.month)[T.11]     0.8009      0.947      0.845      0.398      -1.056       2.658
C(timestamp.dt.month)[T.12]    22.7649   2.61e+04      0.001      0.999   -5.11e+04    5.11e+04
C(is_weekday)[T.True]           0.0392      0.437      0.090      0.929      -0.818       0.896
load                         2.399e-06   5.83e-07      4.115      0.000    1.26e-06    3.54e-06
===============================================================================================
"""

In [7]:
predictions = result.predict(test_data.drop(columns=["curtailment_event"]))
predictions.name = "probability"
predictions = test_data.merge(predictions, left_index=True, right_index=True)

Below is how our test data are actually labeled.

In [8]:
test_data["curtailment_event"].value_counts()

False    282
True      83
Name: curtailment_event, dtype: int64

In [9]:
predictions.query("probability > .8")["curtailment_event"].value_counts().loc[True]

42

Below is a count of our binary classification errors using an arbitrary cutoff probability of .8.  The model predicts the probability a day will have curtailment.

### Model Evaluation

We can calculate a confusion matrix and report back accuracy and precision scores.

In [10]:
true_positives = predictions.query("probability > .8")["curtailment_event"].value_counts().loc[True]
false_negatives = predictions.query("probability > .8")["curtailment_event"].value_counts().loc[False]
true_negatives = predictions.query("probability <= .8")["curtailment_event"].value_counts().loc[False]
false_positives = predictions.query("probability <= .8")["curtailment_event"].value_counts().loc[True]

accuracy = (true_positives+true_negatives)/len(predictions)
precision = true_positives / (true_positives + false_positives)
print(f"Accuracy: {accuracy}; Precision: {precision}")

Accuracy: 0.24383561643835616; Precision: 0.5060240963855421
